In [ ]:
from sklearn.metrics import confusion_matrix
import keras
from keras.layers import LSTM
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import Adam

In [ ]:
def lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes):
    x_train = x_train.reshape(-1, n_step, n_input)
    x_test = x_test.reshape(-1, n_step, n_input)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, n_classes)
    y_test = keras.utils.to_categorical(y_test, n_classes)
    return (x_train, x_test, y_train, y_test)

In [ ]:
def cnn_preprocess(x_train, x_test, y_train, y_test):
    x_train = x_train.reshape(-1, 28, 28, 1)
    x_test = x_test.reshape(-1, 28, 28, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    return (x_train, x_test, y_train, y_test)

In [ ]:
def lstm_model(n_input, n_step, n_hidden, n_classes):
    model = Sequential()
    model.add(LSTM(n_hidden, batch_input_shape=(None, n_step, n_input), unroll=True))
    model.add(Dense(n_classes))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    return model

In [ ]:
def cnn_model():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPool2D(strides=2))
    model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
    model.add(MaxPool2D(strides=2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

In [ ]:
def trainning(model, x_train, y_train, x_test, y_test, 
              learning_rate, training_iters, batch_size):
    adam = Adam(lr=learning_rate)
    model.summary()
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train,
              batch_size=batch_size, epochs=training_iters,
              verbose=1, validation_data=(x_test, y_test))

In [ ]:
def print_confusion_result(x_train, x_test, y_train, y_test, model):
    # get train & test predictions
    train_pred = model.predict_classes(x_train)
    test_pred = model.predict_classes(x_test)
    
    # get train & test true labels
    train_label = y_train
    test_label =  y_test
    
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(10))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(10))
    print(train_result_cm, '\n'*2, test_result_cm)

In [ ]:
def mnist_lstm_main():
    # training parameters
    learning_rate = 0.001
    training_iters = 2
    batch_size = 32

    # model parameters
    n_input = 28
    n_step = 28
    n_hidden = 256
    n_classes = 10

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test, y_train_o, y_test_o = lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes)

    model = lstm_model(n_input, n_step, n_hidden, n_classes)
    trainning(model, x_train, y_train_o, x_test, y_test_o, learning_rate, training_iters, batch_size)
    scores = model.evaluate(x_test, y_test_o, verbose=0)
    print('LSTM test accuracy:', scores[1])
    print_confusion_result(x_train, x_test, y_train, y_test, model)

In [ ]:
def mnist_cnn_main():
    # training parameters
    learning_rate = 0.001
    training_iters = 1
    batch_size = 32

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test, y_train_o, y_test_o = cnn_preprocess(x_train, x_test, y_train, y_test)

    model = cnn_model()
    trainning(model, x_train, y_train_o, x_test, y_test_o, learning_rate, training_iters, batch_size)
    scores = model.evaluate(x_test, y_test_o, verbose=0)
    print('CNN test accuracy:', scores[1])
    print_confusion_result(x_train, x_test, y_train, y_test, model)

In [ ]:
mnist_lstm_main()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 256)               291840    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                110       
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 294,520
Trainable params: 294,520
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
1875/1875 [==============================] - 136s 72ms/step - loss: 0.6389 - accuracy: 0.7833 - val_loss: 0.1272 - val_accuracy: 0.9590
Epoch 2/2
1875/1875 [==============================] - 136s 73ms/step - loss: 0.1013 - accuracy: 0.9

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[5911    0    1    0    0    1    5    0    2    3]
 [   2 6595  110    7    4    1    3   17    1    2]
 [  40    5 5869   12    3    0    5   15    4    5]
 [  36    2   51 5956    0   35    0   20    7   24]
 [  15    4    0    0 5753    1    6   11    2   50]
 [  21    1    6   11    3 5354    7    3    2   13]
 [  49    2    1    1    9   31 5823    0    2    0]
 [   2    6   43    0   13    0    0 6183    3   15]
 [  35   32   17   35   14   57    6   18 5578   59]
 [  17    2    3    2   36    9    1   60    7 5812]] 

 [[ 974    0    0    0    0    2    1    1    2    0]
 [   0 1119    9    1    0    2    1    3    0    0]
 [   8    0 1018    0    0    0    1    5    0    0]
 [  13    0    8  975    0    5    0    6    0    3]
 [   1    0    0    0  962    0    2    2    0   15]
 [   5    0    2    2    0  871    2    3    1    6]
 [  13    2    2    0    3    5  933    0    0    0]
 [   1    1    9    1    0    0    0 1015    0    1]
 [  15    2    4    8    5   14    1    6 

In [ ]:
mnist_cnn_main()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 48)        38448     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 48)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               307456    
_________________________________________________________________
dense_10 (Dense)             (None, 84)               

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[5866    3    4    0   11   10    6    4   10    9]
 [   0 6687    8    2    3    0    0    8   32    2]
 [   1   11 5901    4    4    1    0    8   24    4]
 [   0    0    8 6052    1   34    0    4   15   17]
 [   0    6    2    0 5812    0    0    1    9   12]
 [   0    2    4    5    5 5390    3    0    8    4]
 [   6   22    4    1   30   63 5763    0   29    0]
 [   0   15   34    6   11    0    0 6163   18   18]
 [   3   12    3    3    8   17    0    1 5796    8]
 [   2    6    0    5   57   12    0   17   38 5812]] 

 [[ 968    0    0    0    3    4    0    2    2    1]
 [   0 1127    0    0    0    0    0    0    8    0]
 [   2    1 1022    1    1    0    0    1    4    0]
 [   0    0    2  993    0   10    0    0    4    1]
 [   0    0    1    0  977    0    0    0    3    1]
 [   0    0    0    2    0  887    1    0    0    2]
 [   2    4    0    1   11   10  925    0    5    0]
 [   0    4    6    2    0    0    0 1008    4    4]
 [   2    1    0    0    2    0    0    1 

In [ ]:
from sklearn.metrics import confusion_matrix
import pickle
import keras
from keras.layers import LSTM
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import Adam


def load_pkl(pkl_name):
    # load data from data folder
    with open(pkl_name, 'rb') as f:
        data = pickle.load(f)
    return data

def lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes):
    x_train = x_train.reshape(-1, n_step, n_input)
    x_test = x_test.reshape(-1, n_step, n_input)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = keras.utils.to_categorical(y_train, n_classes)
    y_test = keras.utils.to_categorical(y_test, n_classes)
    return (x_train, x_test, y_train, y_test)

def lstm_model(n_input, n_step, n_hidden, n_classes):
    model = Sequential()
    model.add(LSTM(n_hidden, batch_input_shape=(None, n_step, n_input), unroll=True))
    model.add(Dense(n_classes))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    return model

In [9]:
def train_lstm(model, x_train, y_train, x_test, y_test, 
        learning_rate, training_iters, batch_size):
    adam = Adam(lr=learning_rate)
    model.summary()
    model.compile(optimizer=adam,
        loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train,
        batch_size=batch_size, epochs=training_iters,
        verbose=1, validation_data=(x_test, y_test))

def print_result(data, x_train, x_test, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(x_train)
    test_pred = model.predict_classes(x_test)
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)

def label_18_lstm_main():
    # training parameters
    learning_rate = 0.001
    training_iters = 10
    batch_size = 32

    # model parameters
    n_input = 40
    n_step = 10
    n_hidden = 160
    n_classes = 10
    data = load_pkl('label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl')
    x_train, y_train, x_test, y_test = data['train_gaf'], data['train_label'][:, 0], data['test_gaf'], data['test_label'][:, 0]
    x_train, x_test, y_train, y_test = lstm_preprocess(x_train, x_test, y_train, y_test, n_step, n_input, n_classes)

    model = lstm_model(n_input, n_step, n_hidden, n_classes)
    train_lstm(model, x_train, y_train, x_test, y_test, learning_rate, 
               training_iters, batch_size)
    scores = model.evaluate(x_test, y_test, verbose=0)
    print('LSTM test accuracy:', scores[1])
    print_result(data, x_train, x_test, model)

In [10]:
label_18_lstm_main()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 160)               128640    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1610      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 130,360
Trainable params: 130,360
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
469/469 [==============================] - 9s 17ms/step - loss: 2.0409 - accuracy: 0.2287 - val_loss: 1.3353 - val_accuracy: 0.3852
Epoch 2/10
469/469 [==============================] - 8s 16ms/step - loss: 1.3504 - accuracy: 0.4027 - 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[2109  119  119   82   81  218  137   83   52]
 [ 118 1380    0    2    0    0    0    0    0]
 [ 153    0 1343    0    4    0    0    0    0]
 [ 105   65    0 1245    0    0    0   85    0]
 [ 155    0   51    0 1245    0    0    0   49]
 [  76    5    0    0    0 1394    2   23    0]
 [ 145    2    1    0    4    0 1333    0   15]
 [  62    6    0  223    0   14    0 1195    0]
 [  81    0    2    0  271    0   55    0 1091]] 

 [[715  39  43  24  32  69  32  29  17]
 [ 18 482   0   0   0   0   0   0   0]
 [ 36   0 464   0   0   0   0   0   0]
 [ 30  16   0 436   0   0   0  18   0]
 [ 48   0  25   0 417   0   0   0  10]
 [ 18   0   0   0   0 480   0   2   0]
 [ 42   0   1   0   0   0 452   0   5]
 [ 16   1   0  39   0   6   0 438   0]
 [ 39   0   0   0  68   0  17   0 376]]


In [11]:
from sklearn.metrics import confusion_matrix
import numpy as np
import pickle

from keras import backend as K
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Activation, MaxPool2D


def load_pkl(pkl_name):
    # load data from data folder
    with open(pkl_name, 'rb') as f:
        data = pickle.load(f)
    return data

def get_cnn_model(params):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(10, 10, 4)))
    model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(9, activation='softmax'))
    return model

def train_model(params, data):
    model = get_cnn_model(params)
    model.compile(loss='categorical_crossentropy', optimizer=params['optimizer'], metrics=['accuracy'])
    hist = model.fit(x=data['train_gaf'], y=data['train_label_arr'],
                     batch_size=params['batch_size'], epochs=params['epochs'], verbose=2)
    return (model, hist)

In [12]:
def print_result(data, model):
    # get train & test pred-labels
    train_pred = model.predict_classes(data['train_gaf'])
    test_pred = model.predict_classes(data['test_gaf'])
    # get train & test true-labels
    train_label = data['train_label'][:, 0]
    test_label = data['test_label'][:, 0]
    # confusion matrix
    train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
    test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    print(train_result_cm, '\n'*2, test_result_cm)

In [13]:
PARAMS = {}
PARAMS['pkl_name'] = 'label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl'
PARAMS['classes'] = 9
PARAMS['lr'] = 0.01
PARAMS['epochs'] = 10
PARAMS['batch_size'] = 32
PARAMS['optimizer'] = optimizers.SGD(lr=PARAMS['lr'])

# ---------------------------------------------------------
# load data & keras model
data = load_pkl(PARAMS['pkl_name'])
# train cnn model
model, hist = train_model(PARAMS, data)
# train & test result
scores = model.evaluate(data['test_gaf'], data['test_label_arr'], verbose=0)
print('CNN test accuracy:', scores[1])
print_result(data, model)

Epoch 1/10
469/469 - 7s - loss: 1.2193 - accuracy: 0.5386
Epoch 2/10
469/469 - 7s - loss: 0.5914 - accuracy: 0.7861
Epoch 3/10
469/469 - 7s - loss: 0.4974 - accuracy: 0.8187
Epoch 4/10
469/469 - 7s - loss: 0.4563 - accuracy: 0.8355
Epoch 5/10
469/469 - 7s - loss: 0.4281 - accuracy: 0.8427
Epoch 6/10
469/469 - 7s - loss: 0.4098 - accuracy: 0.8527
Epoch 7/10
469/469 - 7s - loss: 0.3920 - accuracy: 0.8571
Epoch 8/10
469/469 - 7s - loss: 0.3746 - accuracy: 0.8654
Epoch 9/10
469/469 - 7s - loss: 0.3651 - accuracy: 0.8691
Epoch 10/10
469/469 - 7s - loss: 0.3492 - accuracy: 0.8775
CNN test accuracy: 0.8514000177383423


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[2191   61   76  104  125   84   95  176   88]
 [  56 1417    0   24    0    0    0    3    0]
 [  79    0 1413    0    8    0    0    0    0]
 [   9    4    0 1068    0    0    0  419    0]
 [  20    0   21    0 1429    0    1    0   29]
 [  73    0    0    1    0 1391    0   35    0]
 [  60    2    2    0    6    0 1383    0   47]
 [   8    0    0   33    0    3    0 1456    0]
 [  17    0    2    0  248    0   11    0 1222]] 

 [[713  20  34  36  43  39  28  63  24]
 [ 25 455   0  18   0   0   0   2   0]
 [ 20   0 480   0   0   0   0   0   0]
 [ 13   5   0 350   0   0   0 132   0]
 [ 17   0  17   0 452   0   0   0  14]
 [ 44   0   0   0   0 446   0  10   0]
 [ 34   0   0   0   0   0 449   0  17]
 [  4   1   0   1   0   1   0 493   0]
 [  8   0   0   0  68   0   5   0 419]]
